<center>
    <h1> Programación Científica </h1>
    <h1> Actividad Clase 8 </h1>
    <h3> Aldo Berrios Valenzuela  </h3>
    <h3> 201304018-7 </h3>
    <h4> `aldo.berrios.13@sansano.usm.cl` </h4>
</center>

# 1. &nbsp;&nbsp;&nbsp;&nbsp; Preliminares

Importamos las librerías y definimos las funciones que usaremos para el desarrollo de la experiencia:

In [1]:
import numba
import numpy as np
import matplotlib.pyplot as plt
import threading
import multiprocessing
from ipyparallel import Client

def image_plot(img):
    """
    img is an (nx,ny,3) numpy.array
    """
    plt.figure(figsize=(12,12))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

Posteriormente, cargamos la imagen que vamos a utilizar:

In [2]:
img = plt.imread('heic1608b.jpg', format='jpeg')[0:8660,:,:]

Posteriormente, definimos la función que entrega como resultado la matriz kernel gaussiana de $n\times n$:

In [3]:
def gaussian_kernel(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

Definimos la función que aplicará el kernel gaussiano de $3\times 3$ sobre la imagen generada anteriormente.

In [4]:
def apply_kernel(K, vectorized_image):
    result = np.zeros((vectorized_image.shape[0] - 1, vectorized_image.shape[1] - 1, 3))
    for channel in range(3):
        for i in range(1, vectorized_image.shape[0] - 1):        # Recorremos las filas
            for j in range(1, vectorized_image.shape[1] - 1):    # Recorremos las columnas
                result[i, j, channel] = np.sum(vectorized_image[i - 1:i + 2, j - 1:j + 2, channel] * K)
    return result[1:, 1:, :]

Notar que el algoritmo propuesto es secuencial y se paralelizará para el posterior análisis de resultados

# 2. &nbsp;&nbsp;&nbsp;&nbsp; Desarrollo

## 2.1. &nbsp;&nbsp;&nbsp; Pregunta 1

Redefinimos la función `apply_kernel`, de forma tal que en lugar de recibir la imagen completa, sólo recibirá un canal de la imagen:

In [5]:
def apply_kernel(vectorized_image):
    import numpy as np
    K = np.array([[ 0.01134374,  0.08381951,  0.01134374],
       [ 0.08381951,  0.61934703,  0.08381951],
       [ 0.01134374,  0.08381951,  0.01134374]])
    result = np.zeros((vectorized_image.shape[0] - 1, vectorized_image.shape[1] - 1))
    for i in range(1, vectorized_image.shape[0] - 1):        # Recorremos las filas
        for j in range(1, vectorized_image.shape[1] - 1):    # Recorremos las columnas
            result[i, j] = np.sum(vectorized_image[i - 1:i + 2, j - 1:j + 2] * K)
    return result[1:, 1:]

Luego, separamos la imágen original en 3 matrices. Cada una de esta corresponde a un canal RGB:

In [6]:
r_img = img[:, :, 0]
g_img = img[:, :, 1]
b_img = img[:, :, 2]

Posteriormente, instanciamos un cliente

In [7]:
rc = Client()

Luego, creamos un balanceador de carga:

In [8]:
v = rc.load_balanced_view()

Luego, aplicamos el kernel a cada uno de los canales de forma paralela:

In [9]:
args = [r_img, g_img, b_img]
ar = v.map(apply_kernel, args)

Debido a que el código anterior trabaja de forma asincrónica, comprobaremos que todo el cálculo haya terminado con:

In [10]:
ar.ready()

False

Transcurrido un tiempo, comprobamos otra vez:

In [ ]:
ar.ready()

False

### Ahora que tenemos cada canal filtrado, mostramos el resultado por pantalla:

In [ ]:
filtered_image = np.zeros((ar.result()[0].shape[0], ar.result()[0].shape[1], 3))
filtered_image[:, :, 0] = ar.result()[0]
filtered_image[:, :, 1] = ar.result()[1]
filtered_image[:, :, 2] = ar.result()[2]
image_plot(filtered_image)

/anaconda/lib/python3.6/site-packages/matplotlib/cm.py:264: RuntimeWarning: overflow encountered in multiply
  xx = (xx * 255).astype(np.uint8)


In [ ]:
ar.ready()